In [1]:
# Importing libraries
import pandas as pd
import numpy

In [24]:
# Read csv file into a pandas dataframe

# possible missing values which could be found in Data
missing_values = ["n/a", "na", "--"]

# sep = sepreation between different values in data file
# in CSV(comma sepreated values) files, values are sepreated by ','
# loading CSV file & converting all missing values to NAN
df = pd.read_csv("hepatitis.csv", na_values = missing_values, sep = ",")      # converting all empty as NAN

In [35]:
# dropping all the rows which contain any NAN values
df = df.dropna()

In [36]:
## replacing all missing values
## replacing all NAN values with 0 here
#df.fillna(0)

In [37]:
# At this point data is in logical form
# to being able to predict it should be in counting formate
# Main goal is to count all the different values which could be required
# Required Processed Data Arrangement is
#
# {
#     statusValueOne:{
#                 counts for only row containing statusValueOne
#                 symptom1: symptom1 count
#                 symptom2: symptom2 count
#                 .
#                 .
#                 .
#                 symptomN: symptomN count
#                 }
#
#     statusValueTwo:{
#                 counts for only row containing statusValueOne
#                 symptom1: symptom1 count
#                 symptom2: symptom2 count
#                 .
#                 .
#                 .
#                 symptomN: symptomN count
#                 }
# }

In [39]:
# finding the diamentions of the data fram
(rows, columns) = df.shape
# rows = number of rows in data fram
# columns = number of columns in data fram

In [81]:
# let say data is arranged such a way
# all columns are different symptoms(causes) and status(effects)
# it is required to sepreate them

In [84]:
# sepreating symptoms and status
#
# statusColumns = all the columns which contain disease status(effects)
# symptomsColumns = all the columns which contain all the symptoms(causes)
#
# SYNTEX:
# statusColumns = list(range(from, upTo))
# symptomsColumns = list(range(from, upTo))
#
# NOTE: upTo column will also be included

# getting all the columns to be selected
statusColumns = list(range(0, 1))
symptomsColumns = list(range(1, 20))

# sepreating data fram into status & symptoms
status = df.iloc[:, statusColumns]
symptoms = df.iloc[:, symptomsColumns]

In [96]:
status

,class
1,2.0
3,2.0
5,1.0
6,2.0
7,2.0
...,...
136,2.0
137,1.0
138,2.0
139,2.0


In [103]:
from collections import defaultdict

In [104]:
# getting all the unique value in status columns
# so could be counted
# status[status.columns[0]].unique() = list of all the unique values in status columns
uniqueStatusValues = status[status.columns[0]].unique()

# arranged data structure to store all the counts
processedData = defaultdict(dict)

# sepreating data on the bases of uniqueStatusValues & arranging
for eachStatusColumn in statusColumns:
    # for every selected status column
    for eachUniqueValue in uniqueStatusValues:
        processedData[eachUniqueValue] = df[eachStatusColumn].count(eachUniqueValue)

KeyError: 0

In [98]:
myData = {}

In [99]:
myData[0] = 0

In [100]:
myData

{0: 0}

In [101]:
myData[0] = 1

In [102]:
myData

{0: 1}